Lo primero que hacemos es dividir el dataset original en dos partes. 
<BR>

i) Incluye los nulos (10.415) que será donde hagamos las prediciones.
<BR>

ii) No incluye los nulos (21.229) y será el que utilicemos para entrenar el modelo.  


In [7]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('data/data.csv')

data_copia = data.copy()

# Crear un DataFrame con filas que contienen NaN en 'x_e_out [-]'

df_nan = data_copia[data_copia['x_e_out [-]'].isnull()]

# Crear un DataFrame sin filas que contienen NaN en 'x_e_out [-]'

df_sin_nan = data_copia[~data_copia['x_e_out [-]'].isnull()]

print("data_copia: ", data_copia.shape)
print("Data con los NUN:", df_nan.shape)
print("Data sin los NUN: ", df_sin_nan.shape)


data_copia:  (31644, 10)
Data con los NUN: (10415, 10)
Data sin los NUN:  (21229, 10)


In [8]:
# Calcular la media de la columna "x_e_out [-]"
media_x_e_out = df_sin_nan['x_e_out [-]'].mean()

print("Media de x_e_out [-]:", media_x_e_out)

Media de x_e_out [-]: -0.0004528625936219324


In [13]:
# Realizar la codificación one-hot en la columna "author" y "geometry". 

data_encoded = pd.get_dummies(df_sin_nan, columns=['author', "geometry"])

data_encoded.head(2)

,id,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2],author_Beus,author_Inasaka,...,author_Kossolapov,author_Mortimore,author_Peskov,author_Richenderfer,author_Thompson,author_Weatherhead,author_Williams,geometry_annulus,geometry_plate,geometry_tube
0,0,7.0,3770.0,0.1754,NaN,10.8,432.0,3.6,0,0,...,0,0,0,0,1,0,0,0,0,1
1,1,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2,0,0,...,0,0,0,0,1,0,0,0,0,1


# Probamos el modelo de Regresión lineal para hacer el baseline


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Reemplazar los valores faltantes con la media de la columna correspondiente
data_encoded.fillna(data_encoded.mean(), inplace=True)

# Separar las características (X) y la variable objetivo (y)
X = data_encoded.drop('x_e_out [-]', axis=1)
y = data_encoded['x_e_out [-]']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto Scaler y ajustarlo a los datos de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo de regresión lineal
modelo_Regresión= LinearRegression()
modelo_Regresión.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba

y_pred = modelo_Regresión.predict(X_test_scaled)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calcular el R cuadrado
r2 = r2_score(y_test, y_pred)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r2)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)


Root Mean Squared Error (RMSE): 0.08657446870627104
R-squared: 0.2621147584047059
Mean Squared Error (MSE): 0.007495138631773105
Mean Absolute Error (MAE): 0.06298538171593379


# Ponemos en el mismo formato el dataset NAN y hacemos la predicción. 

In [13]:
# Realizar la codificación one-hot en la columna "author" y "geometry"

data_encoded_NUN = pd.get_dummies(df_nan, columns=['author', "geometry"])

# Reemplazar los valores faltantes con la media de la columna correspondiente
data_encoded_NUN.fillna(data_encoded_NUN.mean(), inplace=True)

# Separar las características (X_new) del nuevo DataFrame
X_new = data_encoded_NUN.drop('x_e_out [-]', axis=1)

# Escalar los datos del nuevo DataFrame utilizando el mismo objeto Scaler ajustado previamente
X_new_scaled = scaler.transform(X_new)

# Realizar la predicción en el nuevo DataFrame
y_new_pred = modelo_Regresión.predict(X_new_scaled)

# Agregar los valores predichos al nuevo DataFrame
data_encoded_NUN['x_e_out [-]'] = y_new_pred

# Imprimir el nuevo DataFrame con los valores predichos

formato_subm = data_encoded_NUN[['id', 'x_e_out [-]']]

# Convertir el DataFrame seleccionado a CSV
formato_subm.to_csv('Submission_4.csv', index=False)
